# 自動機械学習 AutoML による与信管理モデル
過去の融資履歴データを用いて、ローンの履行/不履行を予測する与信管理モデルを構築します。

<img src="https://azurecomcdn.azureedge.net/cvt-233147754f3cbba3c733e98527f90b7173b78e5c437a095e31cb4f5ec7d35283/images/page/services/machine-learning/automatedml/learning-models.gif?637587687582664499">

※ 製品ドキュメント
- [Python で自動 ML の実験を構成する](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-configure-auto-train)
- [解釈可能性: 自動機械学習のモデルの説明 (プレビュー)](https://docs.microsoft.com/ja-JP/azure/machine-learning/how-to-machine-learning-interpretability-automl)

## 0. 確認事項
- [ ]  カーネル : `Python 3.6 - Azure ML` が選択されていること
- [ ]  Dataset : hmeq の dataset が Azure ML studio から登録済みであること

## 1. 事前準備
### Azure ML Python SDK のインポート
Azure Machine Learning の Python SDKをインポートします

In [ ]:
from azureml.core import Workspace, Experiment, Dataset

In [ ]:
import azureml
print(azureml.core.VERSION)

### Azure ML Workspace との接続
Azure Machine Learning との接続を行います。Azure Active Directory の認証が必要です。

In [ ]:
ws = Workspace.from_config()

### 実験名の設定
実験 Experimentの機能はメトリックやログファイルなどを管理します。

In [ ]:
experiment_name = 'automl-hmeq-python'
experiment = Experiment(ws, experiment_name)

### 学習データの準備
Azure Machine Learning の Compute Cluster 上でモデル学習をするために、Azure Machine Learning Dataset の形式でデータを定義します。<br>


In [ ]:
titanic_ds = Dataset.get_by_name(ws, name='hmeq', version=4)
titanic_df = titanic_ds.to_pandas_dataframe() # Azure ML Dataset -> Pandas DataFrame

列名が正常に表示されていることを確認します。<br>
※ 列名が一番上に表示されない場合、Azure ML studio から Dataset を登録する際、列見出しを **最初のファイルのみにヘッダーがある** を選択して再度 Dataset を登録します。

In [ ]:
titanic_df.head()

### 学習データとテストデータ (モデル解釈用) に分割
95% のデータをモデル学習に利用します。自動機械学習では交差検証 (クロスバリデーション) が実行されます。

In [ ]:
train_dataset, test_dataset = titanic_ds.random_split(0.95, seed=12345)

### 計算環境 Compute Cluster の設定
Azure Machine Learning は高速にモデル学習を高速に回す計算環境 Machine Learning Compute が利用できます。

In [ ]:
from azureml.core import ComputeTarget
compute_target = ComputeTarget(ws, 'cpuvnet-clusters')

## 2. 自動機械学習 AutoML
### 事前設定
自動機械学習の事前設定を行います。AutoMLConfig で事前設定をします。

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.featurization import FeaturizationConfig

In [ ]:
titanic_df.columns

In [ ]:
num_columns = ['融資依頼金額', '未払担保金額', '現在資産価値', '勤務年数','信用調査会社問い合わせ数', '延滞トレードライン数', '最も古いトレードラインの月齢', '最近のクレジットの問い合わせ数','トレードラインの数', '債務対所得の割合']
featurization_config = FeaturizationConfig()
for col in num_columns:
    featurization_config.add_column_purpose(col, 'Numeric')

In [ ]:
label = "不履行フラグ"

automl_settings = {
    "iteration_timeout_minutes": 5, # 各試行の最大実験時間
    "iterations": 5,  # 試行する機械学習パイプラインの数
    "n_cross_validations": 5,
    "primary_metric": 'accuracy', # 精度指標
}

automl_config = AutoMLConfig(task = 'classification', # regression, forecasting
                             training_data = train_dataset,
                             label_column_name = label,                             
                             compute_target=compute_target,
                             featurization=featurization_config,
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config, show_output = True)

### 結果確認
Notebook で利用できる Widget で実験の様子を確認します。

In [ ]:
# Widget で結果確認
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run

## 3. モデル解釈
オープンソースの InterpretML を利用したモデル解釈機能です。Azure Machine Learning の自動機械学習 AutoML が開発されたモデルの解釈も可能です。

### Explainer の作成
AutoML で開発されたモデルは Azure ML Python SDK を利用して Explainer を生成します。

In [ ]:
# Pandas Dataframe に変換
train_df = train_dataset.to_pandas_dataframe()
test_df = test_dataset.to_pandas_dataframe()

In [ ]:
from azureml.train.automl.runtime.automl_explain_utilities import automl_setup_model_explanations

automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, 
                                                             X=train_df.drop([label], axis=1), 
                                                             X_test=test_df.drop([label], axis=1), 
                                                             y=train_df[label].values, 
                                                             task='classification')

In [ ]:
from azureml.interpret import MimicWrapper

# Initialize the Mimic Explainer
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator,
                         explainable_model=automl_explainer_setup_obj.surrogate_model, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes,
                         explainer_kwargs=automl_explainer_setup_obj.surrogate_model_params)

### Engineered Explanation
特徴量エンジニアリングで生成された変数を用いたモデル解釈を行います。

In [ ]:
# Compute the engineered explanations
engineered_explanations = explainer.explain(['local', 'global'],get_raw=False,
                                            eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(engineered_explanations, 
                     automl_explainer_setup_obj.automl_estimator, 
                     datasetX=automl_explainer_setup_obj.X_test_transform)

### Raw Explanation
特徴量エンジニアリング前の変数を用いたモデル解釈を行います。

In [ ]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(raw_explanations, 
                     automl_explainer_setup_obj.automl_pipeline, 
                     datasetX=automl_explainer_setup_obj.X_test_raw)

#### Responsible-AI-Widgets による可視化
最新のライブラリでは [microsoft/responsible-ai-widgets](https://github.com/microsoft/responsible-ai-widgets) で可視化を行います。

In [ ]:
from raiwidgets import ExplanationDashboard

ExplanationDashboard(raw_explanations, 
                     automl_explainer_setup_obj.automl_pipeline, 
                     dataset=automl_explainer_setup_obj.X_test_raw)